In [1]:
from transformers import AutoModel, AutoTokenizer

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
df = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/notebook/data/openj9/merged_data_dated_cleaned.csv")

In [5]:
df.head()

,Unnamed: 0,issue_number,issue_url,issue_title,issue_body,issue_state,creator,comments,assignees,labels,created_at,closed_at
0,9,10,https://github.com/eclipse-openj9/openj9/pull/10,Update README.md with proper AdoptOpenJDK link,"Also remove the the coming soon reference, sin...",closed,pshipton,NaN,charliegracie,comp:doc,2017-09-14T12:24:25Z,2017-09-14T12:42:15Z
1,14,15,https://github.com/eclipse-openj9/openj9/pull/15,Refactor and clean up CodeCacheManager,- Change invocation parameters of superclass c...,closed,lmaisons,<comment><user>lmaisons</user><body>This has a...,0xdaryl,comp:jit,2017-09-14T14:30:51Z,2017-09-27T20:01:54Z
2,16,17,https://github.com/eclipse-openj9/openj9/pull/17,WIP Add support for function pointers in Panama,When invoking a native function (with Panama) ...,closed,tajila,<comment><user>tajila</user><body>Thanks for t...,DanHeidinga,"comp:vm, project:panama",2017-09-14T16:10:32Z,2019-12-04T15:50:33Z
3,21,22,https://github.com/eclipse-openj9/openj9/pull/22,fix illegal-access behaviour,fix illegal-access behaviour\r\n\r\nThis is th...,closed,tajila,<comment><user>DanHeidinga</user><body>@tajila...,DanHeidinga,"jdk9, comp:vm",2017-09-14T20:53:59Z,2017-09-15T14:23:40Z
4,23,24,https://github.com/eclipse-openj9/openj9/issue...,Enable a travis.ci-based PR build for linux x8...,Build a basic PR build based off the linux_x86...,closed,DanHeidinga,<comment><user>mgaudet</user><body>This has be...,DanHeidinga,comp:build,2017-09-15T04:24:17Z,2017-11-09T14:15:08Z


In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [9]:
encoder = AutoModel.from_pretrained("bert-base-uncased")

In [36]:
issues = df["issue_body"].to_list()

In [38]:
issues

['Also remove the the coming soon reference, since the OpenJ9 build is\r\nlive now.\r\n\r\nSigned-off-by: Peter Shipton <Peter_Shipton@ca.ibm.com>',
 '- Change invocation parameters of superclass constructor, removing the\r\nTR_FrontEnd parameter and replacing it with a TR::RawAllocator.\r\n\r\n- Add a TR::RawAllocator parameter to the J9::CodeCacheManager\r\nconstructor.\r\n\r\n- Add a TR_FrontEnd field in the J9::CodeCacheManger to replace the one\r\nremoved from the OMR layer.\r\n\r\n- Remove the implementations of getMemory and freeMemory methods as they\r\nare now satisfied by the OMR::CodeCacheManager.\r\n\r\nSigned-off-by: Luc des Trois Maisons <lmaisons@ca.ibm.com>',
 "When invoking a native function (with Panama) that takes in a function\r\npointer (ie. qsort), the groveller generates a java abstract method that\r\ntakes in a functional interface with java carrier types that match the\r\nnative types for the function.\r\n\r\nIn the native binding stage the 'generateFunctionPoi

In [43]:
embeddings = []

In [48]:
toks = [tokenizer(str(issue), max_length=512, truncation=True, return_tensors="pt") for issue in issues]

In [49]:
import torch

In [50]:
toks[0]

{'input_ids': tensor([[ 101, 2036, 6366, 1996, 1996, 2746, 2574, 4431, 1010, 2144, 1996, 2330,
         3501, 2683, 3857, 2003, 2444, 2085, 1012, 2772, 1011, 2125, 1011, 2011,
         1024, 2848, 2911, 2669, 1026, 2848, 1035, 2911, 2669, 1030, 6187, 1012,
         9980, 1012, 4012, 1028,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [51]:
with torch.no_grad():
    for tok in toks:
        enc = encoder(input_ids=tok["input_ids"], attention_mask=tok["attention_mask"])["pooler_output"].squeeze(0).numpy()
        embeddings.append(enc)

In [52]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data, NUM_CLUSTERS = 15):

    X = np.array(data['emb'].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

In [53]:
gg = pd.DataFrame()

In [54]:
gg["emb"] = embeddings

In [56]:
clustering_question(gg)

(                                                    emb  cluster  \
 0     [-0.76925015, -0.60363615, -0.976523, 0.642815...        5   
 1     [-0.65502864, -0.48095375, -0.98893505, 0.5079...        1   
 2     [-0.4777021, -0.29245302, -0.8908527, 0.217813...       12   
 3     [-0.45994952, -0.40906882, -0.8705125, 0.13394...       12   
 4     [-0.68880534, -0.5017815, -0.9215463, 0.475670...       10   
 ...                                                 ...      ...   
 5234  [-0.82588685, -0.39264145, -0.91686124, 0.6349...        8   
 5235  [-0.704515, -0.5250255, -0.958367, 0.5076453, ...        5   
 5236  [-0.83206236, -0.3980099, -0.8166663, 0.612540...        8   
 5237  [-0.68001384, -0.303564, -0.78805923, 0.304495...       13   
 5238  [-0.51251864, -0.5658058, -0.95697385, 0.24223...        6   
 
                                                centroid  
 0     [-0.7305392, -0.5464955, -0.97699255, 0.607061...  
 1     [-0.56067556, -0.58351266, -0.98799676, 0.432